## Практическое задание

<ol>
    <li>Попробовать улучшить точность распознования образов cifar 10 сверточной нейронной сетью, рассмотренной на уроке. Приложить анализ с описанием того, что улучшает работу нейронной сети и что ухудшает.
    </li>
    <li>Описать также в анализе какие необоходимо внести изменения  в получившуюся у вас нейронную сеть если бы ей нужно было работать не с cifar10, а с MNIST, CIFAR100 и IMAGENET.
    </li>
</ol>

In [33]:
from __future__ import print_function
import keras # расскоментируйте эту строку, чтобы начать обучение
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import os
import time

start_time = time.clock()

# установка параметров нейросети
batch_size = 32
num_classes = 10
epochs = 15
data_augmentation = True
num_predictions = 30
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# разделение тренировочной и тестовой выборки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(100, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
# model.add(BatchNormalization())

# model.add(Conv2D(128, (3, 3)))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# # model.add(Conv2D(128, (3, 3)))
# # model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.3))


model.add(Conv2D(200, (3, 3), padding='same'))
model.add(Activation('relu'))
# model.add(Conv2D(64, (3, 3), padding='same'))
# model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))


model.add(Conv2D(100, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
# model.add(Conv2D(32, (3, 3)))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))



# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(50))
model.add(Activation('relu'))
# model.add(BatchNormalization())
model.add(Dropout(0.3))


model.add(Dense(num_classes))
model.add(Activation('softmax'))

# инициализация RMSprop optimizer
# opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
#opt= keras.optimizers.Adam(learning_rate=lr_schedule)
opt = keras.optimizers.Adam(learning_rate=0.0001)

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation в реальном времени')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=0, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
#         rescale=0.01,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)

    # запуск data augmentation через fit
    #datagen.fit(x_train)

    # запуск data augmentation через fit_generator
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

print ("Время выполнения {:g} минут".format((time.clock() - start_time)/60))

# сохранение модели и весов
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_name)
    model.save(model_path)
    print('сохранить обученную модель как %s ' % model_path)

# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

/Users/daniilstv/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  # This is added back by InteractiveShellApp.init_path()


x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры
Использование data augmentation в реальном времени
Epoch 1/15
1563/1563 [==============================] - 822s 526ms/step - loss: 1.9065 - accuracy: 0.3114 - val_loss: 1.4637 - val_accuracy: 0.4694
Epoch 2/15
1563/1563 [==============================] - 816s 522ms/step - loss: 1.6060 - accuracy: 0.4221 - val_loss: 1.3620 - val_accuracy: 0.5186
Epoch 3/15
1563/1563 [==============================] - 816s 522ms/step - loss: 1.4406 - accuracy: 0.4866 - val_loss: 1.1431 - val_accuracy: 0.5974
Epoch 4/15
1563/1563 [==============================] - 816s 522ms/step - loss: 1.3068 - accuracy: 0.5400 - val_loss: 1.0784 - val_accuracy: 0.6261
Epoch 5/15
1563/1563 [==============================] - 820s 525ms/step - loss: 1.2246 - accuracy: 0.5684 - val_loss: 1.0037 - val_accuracy: 0.6521
Epoch 6/15
1563/1563 [==============================] - 811s 519ms/step - loss: 1.1633 - accuracy: 0.5957 - val_loss: 1.0584

/Users/daniilstv/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:138: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


10000/10000 [==============================] - 47s 5ms/step
Test loss: 0.7149908617019654
Test accuracy: 0.7562999725341797


Базовые показатели модели на уроке:
Test loss: 1.528359656906128
Test accuracy: 0.4399999976158142


## Проведены настройки модели:
Первоначально выкрутил ручки побольше и получил accuracy - 0.13. Явное переобучение. 
Перебором понял, что проблема была в дополнительных полносвязных слоях. Оставил один.
После чего тюнил модель. Добрался до 0.55-0.58. В итоге упростил модель - сократил количество слоёв и поставил 15 эпох.

Впроцессе тюнинга пробовал разные варианты:
* Увеличены эпохи до 10. После 11 accuracy переставал расти.
* Увеличил сходимость за счет повышения learning_rate до 0.001, что привело к снижению точности - вернул на 0.0001
* Сократил помехи Dropout до 0.1. Затем расставил разные значения в зависимости от кол-ва нейронов в слое - от 0.3 до 0.1
* Добавил сверточных слоёв с уменьшением нейронов (128, 64, 32) для повышения абстракции, разбавил их макспулингом. 
* Сначала добавил два полносвязные слои (128 и 64) перед выходом. Полносвязные слои серьёзно переобучают, снижая результат до 0.15. Оставил один слой 64.
* Увеличил num_predictions до 50
* Увеличил batch_size до 64
* Добавил в слои нормализацию BatchNormalization()
* Добавил в искажения data augmentation масштабирование изображения на 0.01 - https://keras.io/api/preprocessing/image/


Использовал работы 

https://github.com/DmitriyFedorov-git/GeekBrains/blob/c89e20e1c417b42f4e21d68fc10e549f4aa7a334/Intro_NN_homework_4_CNN.ipynb
https://github.com/rajulun/Introduction_to_neural_networks/blob/1bc070f7c9c32fd0e4ebe91856e0713188fafd94/Introduction%20to%20neural%20networks%20Lesson4.ipynb
https://github.com/mindblower5000/neural-nets/blob/4a12ab3f518da7826b98deff1b82b3747d5730c1/hw4.ipynb 


#### Описать также в анализе какие необоходимо внести изменения в получившуюся у вас нейронную сеть если бы ей нужно было работать не с cifar10, а с MNIST, CIFAR100 и IMAGENET.


Во всех случая параметр выходного слоя num_classes должен соответствовать количеству выявляемых категорий.


Для более сложеных сетов CIFAR100 и IMAGENET потребуется увеличение масштаба внутренних слоёв.
Я бы сделал постепенное повышение уровня абстракции от слоя к слою. 

Для MNIST можно упростить модель: сократить слои, убрать сдвиг изображения. Этому датасету достаточно минимальных настроек сети. Входящий слой должен быть равным матрице пикселей (28 * 28). Для цветных картинок добавляется третье измерение RGB равное 3.

## Дополнительные материалы

<ol>
    <li>https://keras.io/layers/convolutional/</li>
    <li>https://keras.io/layers/pooling/</li>
    <li>https://keras.io/preprocessing/image/</li>
</ol>